# Shifted-tilted Tophat at Gaussian interference at waist

Tophat incoming with shift then tilt ( to 1st-order in both):
\begin{align*}
\sum u_{mn} \rightarrow_{shift,tilt} &
    \sum u_{mn} (1 + \frac{2 a x}{w_0^2})(1+ i k \alpha x)
    \\
    =&
    \sum u_{mn} (1 + \frac{2 a }{w_0^2} x + i k \alpha x + i \frac{2 k \alpha a}{w_0^2} x^2 )
\end{align*}

For $x$:

\begin{equation}
x u_{n,m} = \frac{w_0}{2}
\Big[
	(1 - i \frac{z}{z_R})
	\sqrt{n+1} u_{n+1,m}
	+
	\sqrt{n}
	(1+i \frac{z}{z_R})
	u_{n-1,m}
\Big]
\end{equation}

For $x^2$:

\begin{equation}
x^2 u_{n,m} = \frac{w_0^2}{4}
\Big[
	(1 - i \frac{z}{z_R})^2
	\sqrt{(n+1)(n+2)} u_{n+2,m}
	+
	(2n+1)
	(1+ (\frac{z}{z_R})^2)
	u_{n,m}
	+
	\sqrt{n(n-1)}
	(1+i \frac{z}{z_R} )^2
	u_{n-2,m}
\Big]
\end{equation}

## Imports, global constants

In [1]:
import PauLisa as pl, numpy as np, matplotlib.pyplot as plt
from scipy.special import erfi as erfi

from scipy import integrate
from mpmath import quadgl

import scipy.io

pi=np.pi

## Tophat coefficients from AW tophat file

In [2]:
mat = scipy.io.loadmat('Top_hat_for_paul.mat')
for x in mat:
  print(x)

__header__
__version__
__globals__
coeftop
read_me_paul
wb


In [3]:
print(mat['read_me_paul'])
print(mat['wb'])

coef=mat['coeftop'].ravel()

['wb is the waist to use for tophat basis with tophat radius 1mm,coeftop are the mode coefficients.']
[[0.00023067]]


Using simtools mode indexing in Python:

$
m = \frac{(N+1)(N+2)}{2}- (A_{python} + 1)
$

$
\rightarrow A_{python} =  \frac{(N+1)(N+2)}{2} + 1 -m 
$

$
n = A_{python} - \frac{N(N+1)}{2} = N-m
$

$
\rightarrow A_{python} = N-m + - \frac{N(N+1)}{2} 
$

$
N = floor( \frac{\sqrt{8A_{python}+1} - 1}{2} ) = ceil(\frac{\sqrt{9+8A_{python}} - 3}{2} )
$

## Get Tophat and Gaussian beams from modes (after converting from simtools notation)

In [4]:
def N_f(A):
    
    res = np.floor((np.sqrt(8*A+1)-1)/2)
    
    #res = (np.sqrt(9+8*A)-3)/2
    return(res)

def m(N,A):
    res = (N+1)*(N+2)/2 - (A+1)
    return(res)

#def n(N,A):
#    m= (N+1)*(N+2)/2 - (A+1)
#    res = N-m
#    return(res)
def n(N,A):
    res = A - (N*(N+1)/2)
    return(res)

NumberModes = int(len(coef))
listModesN = [None] * NumberModes
listModesM = [None] * NumberModes
listModesC = [None] * NumberModes



#for i in range(len(coef)):
#    A=i
#    N= N_f(A)
#
#    if (m(N,A)%2 == 0) & (n(N,A)%2 == 0):
#        print(coef[A], '\t\t\t' , m(N,A), ',' , n(N,A))

for i in range(NumberModes):
    A=i
    N= N_f(A)
    listModesN[i] = int(m(N,A))
    listModesM[i] = int(n(N,A))
    listModesC[i] = coef[i]
    
    

In [5]:
plane = pl.Plane(-.5e-2, .5e-2, 300, -.5e-2, .5e-2, 100)
params = pl.Params(1064e-9,1e-3,0.00023067)

modes_tophat = pl.create_modes(listModesM,listModesN,listModesC,NumberModes)
modes_gauss = pl.modes((0,0,1))

## Optical Input parameters, and tilt/shift

In [6]:
#spot, wavelength, shift
w= 1e-3
lam= 1064e-9
a = 100e-6  #100 micron shift
k = 2*pi/lam
#z0 = 0.00023067 # for tophat with 1e-3 radius

#alpha(10nrad)
alpha_min=-500e-6
alpha_max=500e-6
num_points = 2
alpha = np.linspace(alpha_min,alpha_max,num = num_points)


## Integrate over tophat and gaussian (iterate through alphas)

\begin{equation}
C_{nmn'm'}^{R} = \int_{0}^{\infty} dx\int_{-\infty}^{\infty} dy \; U_{00(HG00)}^*(x,y,z_0) U_{nm(tilt-shift Tophat)}(x,y,z_0')
\end{equation}

In [ ]:
def integ_r(alpha):
    for i in range (len(alpha)):
        int_r = [0]*len(alpha) #integral right
        
        print(i) #progress check
        
        #define integral = dydx for given plane, conjugate(HG00)*tophat
        integral_amp = lambda y, x: (np.conjugate(pl.amplitude(params,x,y,0,modes_gauss))
                             *(pl.amplitude_case2(params,x,y,0.00023067,modes_tophat,a,alpha[i])))  
        int_r[i] = quadgl(integral_amp, [plane.ymin,plane.ymax],[0,plane.xmax])
        return(int_r)

c_r = integ_r(alpha) #results for right,left integrals


0


\begin{equation}
C_{nmn'm'}^{L} = \int_{-\infty}^{0} dx\int_{-\infty}^{\infty} dy \; U_{00(HG00)}^*(x,y,z_0) U_{nm(tilt-shift Tophat)}(x,y,z_0')
\end{equation}

In [ ]:
def integ_r(alpha):
    for i in range (len(alpha)):
        int_l = [0]*len(alpha) #integral left
        
        print(i) #progress check
        
        #define integral = dydx for given plane, conjugate(HG00)*tophat
        integral_amp = lambda y, x: (np.conjugate(pl.amplitude(params,x,y,0,modes_gauss))
                             *(pl.amplitude_case2(params,x,y,0.00023067,modes_tophat,a,alpha[i])))
        int_l[i] = quadgl(integral_amp, [plane.ymin,plane.ymax],[plane.xmin,0])
        return(int_l)

c_l = integ_r(alpha) #results for right,left integrals


## Phases from integral results (right and left), DWS & LPS from phases

In [7]:
phase_r = cm.phase(c_r) #phase right side PD
phase_l = cm.phase(c_l) #phase left side PD

dws = .5*(phase_r-phase_l) #DWS signal phase
lps = .5*(phase_r+phase_l) #LPS signal phase

## Plot GWS,LPS

In [ ]:
plt.figure(figsize=(9,6))

plt.plot(alpha, dws, label='DWS') 

plt.legend(loc='lower right')

plt.xlabel(r'Intensity [a.u.], $\alpha\;$[' + xlab +'rad]', fontsize=15) 
plt.ylabel(r'DWS Signal Phase, $ [' + ylab + ']$', fontsize=15) 
plt.title(r'Gapless HPPD: Shift-Tilted Tophat-HG00 at Waist (shift ='+str(a)+'$\mu m$,)') 
plt.grid() 
#plt.savefig("tilt-shift-waist-2tilt.pdf")

In [ ]:
plt.figure(figsize=(9,6))

plt.plot(alpha, lps, label='LPS') 

plt.legend(loc='lower right')

plt.xlabel(r'Intensity [a.u.], $\alpha\;$[' + xlab +'rad]', fontsize=15) 
plt.ylabel(r'DWS Signal Phase, $ [' + ylab + ']$', fontsize=15) 
plt.title(r'Gapless HPPD: Shift-Tilted Tophat-HG00 at Waist (shift ='+str(a)+'$\mu m$,)') 
plt.grid() 
#plt.savefig("tilt-shift-waist-2tilt.pdf")